In [1]:
import numpy as np
import glob
import os
from models.JetPointNet import PointNetSegmentation
from jets_train import load_data_from_npz

# Setup
os.environ['CUDA_VISIBLE_DEVICES'] = ""  # Disable GPU
model_path = "saved_model/PointNetModel.keras"
# Change TEST_DIR based on the task
# For classification
# TEST_DIR = '/path/to/classification/test_data'
# For energy prediction
TEST_DIR = '/data/mjovanovic/jets/processed_files/2000_events_w_fixed_hits/SavedNpz/test'

task = "classification"  # Use "classification" for the first task, "energy" for the second

model = PointNetSegmentation(num_points=859, num_classes=1)
model.load_weights(model_path)

npz_files = glob.glob(os.path.join(TEST_DIR, '*.npz'))



2024-04-05 12:09:37.947126: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9360] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-05 12:09:37.953099: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-05 12:09:37.968794: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1537] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-05 12:09:39.381073: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-05 12:09:52.123904: E tensor

In [2]:
metrics = []
for npz_file in npz_files:
    feats, frac_labels, tot_labels, tot_truth_e = load_data_from_npz(npz_file)

    # for regression
    labels = tot_labels

    # for classification
    labels = frac_labels
    
    predicted_labels = np.squeeze(model.predict(feats), axis=-1)
    
    for sample_index in range(len(feats)):
        current_labels = labels[sample_index]
        current_predicted_labels = predicted_labels[sample_index]
        current_energies = feats[sample_index][:, 4]  # Energy is at index 4

        valid_indices = np.where(current_labels != -1)
        

        if task == "classification":
            # Your original classification logic here
            absolute_errors = np.abs(current_predicted_labels[valid_indices] - current_labels[valid_indices])
            misclassified_energy = np.sum(absolute_errors * current_energies[valid_indices])
            total_energy = np.sum(current_energies[valid_indices])
            metric = misclassified_energy / total_energy if total_energy else 0
        elif task == "energy":
            # New logic for energy prediction
            absolute_errors = np.abs(current_predicted_labels[valid_indices] - current_labels[valid_indices])
            print(max(absolute_errors))
            # Mean Absolute Error for energy predictions
            mae = np.mean(absolute_errors)
            metric = mae

        
        metrics.append(metric)

average_metric = np.mean(metrics)
if task == "classification":
    print(f"Average Misclassification Energy Ratio: {1 - average_metric:.4f}")
elif task == "energy":
    print(f"Mean Properly Classified Energy (Energy Prediction): {1 - average_metric:.4f}")

13/13 [==============================] - 3s 260ms/step
Average Misclassification Energy Ratio: 0.8079
